In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Database connection
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'ecommerce'
DB_USER = 'postgres'
DB_PASSWORD = '18*DhyaneshvaR'

connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(connection_string)

# Load data
users_df = pd.read_sql("SELECT * FROM users WHERE role = 'buyer'", engine)
products_df = pd.read_sql("SELECT * FROM products WHERE is_active = true", engine)
events_df = pd.read_sql("SELECT * FROM events", engine)

# Convert timestamps
events_df['created_at'] = pd.to_datetime(events_df['created_at'])
products_df['created_at'] = pd.to_datetime(products_df['created_at'])

print(f"Data loaded: {len(users_df)} users, {len(products_df)} products, {len(events_df)} events")

Data loaded: 101 users, 104 products, 12728 events


In [5]:
# Calculate user activity metrics
user_features = events_df.groupby('user_id').agg({
    'id': 'count',
    'created_at': ['min', 'max']
}).reset_index()

user_features.columns = ['user_id', 'total_interactions', 'first_interaction', 'last_interaction']

# Calculate active days
user_features['days_active'] = (user_features['last_interaction'] - user_features['first_interaction']).dt.days + 1

# Activity rate (interactions per day)
user_features['activity_rate'] = user_features['total_interactions'] / user_features['days_active']

# Recency (days since last activity)
latest_date = events_df['created_at'].max()
user_features['days_since_last_activity'] = (latest_date - user_features['last_interaction']).dt.days

print("User Basic Features:")
print(user_features.head())
print(f"\nShape: {user_features.shape}")

User Basic Features:
   user_id  total_interactions       first_interaction  \
0        1                  45 2025-11-22 16:25:46.375   
1       13                 197 2025-11-18 15:58:11.640   
2       14                  70 2025-12-03 07:36:26.697   
3       15                 212 2025-11-23 02:58:51.575   
4       16                 108 2025-11-29 02:16:47.360   

         last_interaction  days_active  activity_rate  \
0 2025-12-07 21:34:03.242           16       2.812500   
1 2025-12-07 22:02:05.758           20       9.850000   
2 2025-12-13 08:37:20.639           11       6.363636   
3 2025-12-17 04:17:52.992           25       8.480000   
4 2025-12-13 07:27:56.123           15       7.200000   

   days_since_last_activity  
0                         9  
1                         9  
2                         4  
3                         0  
4                         4  

Shape: (101, 7)


In [6]:
# Count each event type per user
event_type_counts = events_df.pivot_table(
    index='user_id',
    columns='event_type',
    values='id',
    aggfunc='count',
    fill_value=0
).reset_index()

# Rename columns
event_type_counts.columns = ['user_id', 'add_to_cart_count', 'click_count', 'purchase_count', 'view_count']

# Merge with user_features
user_features = user_features.merge(event_type_counts, on='user_id', how='left')

# Calculate conversion metrics
user_features['view_to_click_rate'] = user_features['click_count'] / user_features['view_count']
user_features['click_to_cart_rate'] = user_features['add_to_cart_count'] / user_features['click_count']
user_features['cart_to_purchase_rate'] = user_features['purchase_count'] / user_features['add_to_cart_count']
user_features['overall_conversion_rate'] = user_features['purchase_count'] / user_features['view_count']

# Fill NaN (for users with no purchases/carts)
user_features = user_features.fillna(0)

print("User Conversion Metrics:")
print(user_features[['user_id', 'view_to_click_rate', 'click_to_cart_rate', 'cart_to_purchase_rate', 'overall_conversion_rate']].head(10))

User Conversion Metrics:
   user_id  view_to_click_rate  click_to_cart_rate  cart_to_purchase_rate  \
0        1            0.652174            0.400000               0.166667   
1       13            0.650000            0.430769               0.142857   
2       14            0.833333            0.520000               0.153846   
3       15            0.607143            0.397059               0.185185   
4       16            0.784314            0.375000               0.133333   
5       17            0.720000            0.444444               0.125000   
6       18            0.689655            0.500000               0.100000   
7       19            0.634921            0.550000               0.136364   
8       20            0.671756            0.306818               0.148148   
9       21            0.573643            0.459459               0.147059   

   overall_conversion_rate  
0                 0.043478  
1                 0.040000  
2                 0.066667  
3          

In [16]:
# Get category for each event
events_with_category = events_df.merge(
    products_df[['id', 'category']], 
    left_on='product_id', 
    right_on='id', 
    how='left'
)

# Count interactions per user per category
user_category_counts = (
    events_with_category
    .groupby(['user_id', 'category'])
    .size()
    .reset_index(name='interaction_count')
)

# Get top 3 categories per user
top_categories = (
    user_category_counts
    .sort_values(['user_id', 'interaction_count'], ascending=[True, False])
    .groupby('user_id')
    .head(3)
)

# Collect top categories into a list
user_top_cats = (
    top_categories
    .groupby('user_id')['category']
    .apply(list)
    .reset_index()
)

# Create explicit top-3 columns
user_top_cats['favorite_category_1'] = user_top_cats['category'].apply(lambda x: x[0] if len(x) > 0 else None)
user_top_cats['favorite_category_2'] = user_top_cats['category'].apply(lambda x: x[1] if len(x) > 1 else None)
user_top_cats['favorite_category_3'] = user_top_cats['category'].apply(lambda x: x[2] if len(x) > 2 else None)

user_top_cats = user_top_cats.drop(columns=['category'])

# Category diversity (unique categories per user)
user_category_diversity = (
    events_with_category
    .groupby('user_id')['category']
    .nunique()
    .reset_index(name='category_diversity')
)

# -------------------------
# SAFE MERGES (drop first)
# -------------------------
cols_to_drop = [
    'favorite_category_1',
    'favorite_category_2',
    'favorite_category_3',
    'category_diversity'
]

user_features = user_features.drop(
    columns=[c for c in cols_to_drop if c in user_features.columns]
)

user_features = user_features.merge(
    user_top_cats,
    on='user_id',
    how='left'
)

user_features = user_features.merge(
    user_category_diversity,
    on='user_id',
    how='left'
)

print("📂 User Category Preferences:")
print(
    user_features[
        ['user_id', 'favorite_category_1', 'favorite_category_2', 'favorite_category_3', 'category_diversity']
    ].head(10)
)


📂 User Category Preferences:
   user_id     favorite_category_1 favorite_category_2 favorite_category_3  \
0        1        Food & Beverages            Clothing        Toys & Games   
1       13       Sports & Outdoors       Home & Garden               Other   
2       14             Electronics            Clothing               Other   
3       15  Beauty & Personal Care       Home & Garden          Automotive   
4       16            Toys & Games          Automotive               Books   
5       17  Beauty & Personal Care          Automotive            Clothing   
6       18                Clothing       Home & Garden          Automotive   
7       19                Clothing    Food & Beverages          Automotive   
8       20        Food & Beverages         Electronics               Books   
9       21             Electronics               Other            Clothing   

   category_diversity  
0                   7  
1                   9  
2                   9  
3               

In [17]:
# Get purchases with prices
purchase_events = events_df[events_df['event_type'] == 'purchase'].merge(
    products_df[['id', 'price']], 
    left_on='product_id', 
    right_on='id', 
    how='left'
)

# Calculate price statistics per user
user_price_stats = purchase_events.groupby('user_id')['price'].agg([
    ('avg_purchase_price', 'mean'),
    ('min_purchase_price', 'min'),
    ('max_purchase_price', 'max'),
    ('std_purchase_price', 'std')
]).reset_index()

# Fill NaN for users with no purchases
user_price_stats = user_price_stats.fillna(0)

# Get browsing price stats (from views)
view_events = events_df[events_df['event_type'] == 'view'].merge(
    products_df[['id', 'price']], 
    left_on='product_id', 
    right_on='id', 
    how='left'
)

user_browse_price = view_events.groupby('user_id')['price'].mean().reset_index(name='avg_browsed_price')

# Merge price features
user_features = user_features.merge(user_price_stats, on='user_id', how='left')
user_features = user_features.merge(user_browse_price, on='user_id', how='left')
user_features = user_features.fillna(0)

# Create price bucket preference
def get_price_bucket(price):
    if price < 500:
        return 'budget'
    elif price < 1000:
        return 'mid'
    elif price < 5000:
        return 'high'
    else:
        return 'luxury'

user_features['price_preference'] = user_features['avg_purchase_price'].apply(get_price_bucket)

print("💰 User Price Affinity:")
print(user_features[['user_id', 'avg_purchase_price', 'avg_browsed_price', 'price_preference']].head(10))

💰 User Price Affinity:
   user_id  avg_purchase_price  avg_browsed_price price_preference
0        1              750.00       27523.000000              mid
1       13             3515.50        4708.120000             high
2       14              558.50        9185.366667              mid
3       15             1023.80        5489.901786             high
4       16             1249.00        2997.627451             high
5       17              318.00        7823.480000           budget
6       18             2799.00       25347.551724             high
7       19              887.00        6673.301587              mid
8       20              303.25       12490.374046           budget
9       21             6440.20       20716.426357           luxury


In [24]:
# Aggregate purchase events per user
purchase_users = (
    purchase_events
    .groupby('user_id')
    .agg(
        first_purchase=('created_at', 'min'),
        last_purchase=('created_at', 'max'),
        total_purchases=('created_at', 'count')
    )
    .reset_index()
)

# Days since first and last purchase
purchase_users['days_since_first_purchase'] = (
    latest_date - purchase_users['first_purchase']
).dt.days

purchase_users['days_since_last_purchase'] = (
    latest_date - purchase_users['last_purchase']
).dt.days

# Purchase frequency (purchases per active day)
purchase_users['purchase_days_span'] = (
    (purchase_users['last_purchase'] - purchase_users['first_purchase']).dt.days + 1
)

purchase_users['purchase_frequency'] = (
    purchase_users['total_purchases'] / purchase_users['purchase_days_span']
)

# -------------------------
# SAFE MERGE (drop first)
# -------------------------
cols_to_drop = [
    'days_since_first_purchase',
    'days_since_last_purchase',
    'purchase_frequency',
    'total_purchases'
]

user_features = user_features.drop(
    columns=[c for c in cols_to_drop if c in user_features.columns]
)

user_features = user_features.merge(
    purchase_users[
        ['user_id',
         'total_purchases',
         'days_since_first_purchase',
         'days_since_last_purchase',
         'purchase_frequency']
    ],
    on='user_id',
    how='left'
)

# Fill NaNs ONLY for purchase-related columns
purchase_cols = [
    'total_purchases',
    'days_since_first_purchase',
    'days_since_last_purchase',
    'purchase_frequency'
]

user_features[purchase_cols] = user_features[purchase_cols].fillna(0)

print("User Purchase Frequency & Recency:")
print(
    user_features[
        ['user_id', 'total_purchases', 'days_since_last_purchase', 'purchase_frequency']
    ].head(10)
)


User Purchase Frequency & Recency:
   user_id  total_purchases  days_since_last_purchase  purchase_frequency
0        1              1.0                      24.0            1.000000
1       13              4.0                       9.0            0.235294
2       14              2.0                       5.0            0.200000
3       15              5.0                       0.0            0.200000
4       16              2.0                      11.0            0.250000
5       17              1.0                       4.0            1.000000
6       18              1.0                      22.0            1.000000
7       19              3.0                       1.0            0.300000
8       20              4.0                       2.0            0.400000
9       21              5.0                      14.0            0.625000


In [25]:
# Create user segments based on behavior
def segment_user(row):
    if row['purchase_count'] == 0:
        if row['total_interactions'] >= 100:
            return 'browser'
        else:
            return 'casual'
    elif row['purchase_count'] == 1:
        return 'one_time_buyer'
    elif row['purchase_count'] >= 2 and row['purchase_count'] < 5:
        return 'regular_buyer'
    else:
        return 'power_buyer'

user_features['user_segment'] = user_features.apply(segment_user, axis=1)

print("User Segments Distribution:")
print(user_features['user_segment'].value_counts())

User Segments Distribution:
user_segment
regular_buyer     64
power_buyer       16
one_time_buyer    13
casual             7
browser            1
Name: count, dtype: int64


In [26]:
# Product interaction statistics
product_features = events_df.groupby('product_id').agg({
    'id': 'count',
    'user_id': 'nunique'
}).reset_index()

product_features.columns = ['product_id', 'total_interactions', 'unique_users']

# Event type breakdown for products
product_event_counts = events_df.pivot_table(
    index='product_id',
    columns='event_type',
    values='id',
    aggfunc='count',
    fill_value=0
).reset_index()

product_event_counts.columns = ['product_id', 'add_to_cart_count', 'click_count', 'purchase_count', 'view_count']

# Merge
product_features = product_features.merge(product_event_counts, on='product_id', how='left')

# Merge with product details
product_features = product_features.merge(
    products_df[['id', 'name', 'category', 'price', 'stock', 'created_at']], 
    left_on='product_id', 
    right_on='id', 
    how='left'
)

print("Product Features:")
print(product_features.head())

Product Features:
   product_id  total_interactions  unique_users  add_to_cart_count  \
0           3                 104            35                 16   
1           4                 101            38                 13   
2           5                 109            33                 15   
3           6                  96            35                 12   
4           7                 109            42                 15   

   click_count  purchase_count  view_count  id  \
0           34               0          54   3   
1           33               0          55   4   
2           38               0          56   5   
3           34               0          50   6   
4           36               0          58   7   

                                                name      category     price  \
0    HP VICTUS 15 LAPTOP i7 12th Gen NVIDIA 3050 RTX   Electronics   90000.0   
1  LG Ultragear 24GS65F 24 inch Full HD (1920 X 1...   Electronics   12000.0   
2                  G

In [27]:
# Calculate product conversion rates
product_features['click_through_rate'] = product_features['click_count'] / product_features['view_count']
product_features['add_to_cart_rate'] = product_features['add_to_cart_count'] / product_features['click_count']
product_features['purchase_rate'] = product_features['purchase_count'] / product_features['view_count']

# Fill NaN
product_features = product_features.fillna(0)

# Popularity score (weighted combination)
product_features['popularity_score'] = (
    product_features['view_count'] * 1 +
    product_features['click_count'] * 2 +
    product_features['add_to_cart_count'] * 3 +
    product_features['purchase_count'] * 5
)

# Normalize popularity score (0-100)
product_features['popularity_score_normalized'] = (
    (product_features['popularity_score'] - product_features['popularity_score'].min()) / 
    (product_features['popularity_score'].max() - product_features['popularity_score'].min()) * 100
)

print("Product Conversion Metrics:")
print(product_features[['name', 'view_count', 'click_through_rate', 'purchase_rate', 'popularity_score_normalized']].head(10))

Product Conversion Metrics:
                                                name  view_count  \
0    HP VICTUS 15 LAPTOP i7 12th Gen NVIDIA 3050 RTX          54   
1  LG Ultragear 24GS65F 24 inch Full HD (1920 X 1...          55   
2                  Gucci Men's GG printed Silk Shirt          56   
3                    Gucci Ophidia Mini Shoulder Bag          50   
4       WWE 2K25 | Standard Edition | PlayStation 4           58   
5        Grand Theft Auto V - Premium Edition (PS4)           63   
6  EvoFox One S Universal 3-Mode Wireless Gaming ...          40   
7                       The Alchemist - Paulo Coelho          61   
8  Harry Potter 1–3 Box Set: A Magical Adventure ...          71   
9                             One-Punch Man, Vol. 30          65   

   click_through_rate  purchase_rate  popularity_score_normalized  
0            0.629630       0.000000                    21.327014  
1            0.600000       0.000000                    16.587678  
2            0.6785

In [28]:
# Price bucket
product_features['price_bucket'] = product_features['price'].apply(get_price_bucket)

# Days since product added
product_features['days_since_added'] = (latest_date - product_features['created_at']).dt.days

# Product age category
def get_age_category(days):
    if days < 7:
        return 'new'
    elif days < 14:
        return 'recent'
    else:
        return 'established'

product_features['age_category'] = product_features['days_since_added'].apply(get_age_category)

# Category encoding (will use later)
le_category = LabelEncoder()
product_features['category_encoded'] = le_category.fit_transform(product_features['category'])

print("Product Price & Category Features:")
print(product_features[['name', 'price', 'price_bucket', 'category', 'days_since_added', 'age_category']].head(10))

Product Price & Category Features:
                                                name     price price_bucket  \
0    HP VICTUS 15 LAPTOP i7 12th Gen NVIDIA 3050 RTX   90000.0       luxury   
1  LG Ultragear 24GS65F 24 inch Full HD (1920 X 1...   12000.0       luxury   
2                  Gucci Men's GG printed Silk Shirt  104064.0       luxury   
3                    Gucci Ophidia Mini Shoulder Bag   84360.0       luxury   
4       WWE 2K25 | Standard Edition | PlayStation 4     2899.0         high   
5        Grand Theft Auto V - Premium Edition (PS4)     1977.0         high   
6  EvoFox One S Universal 3-Mode Wireless Gaming ...    1599.0         high   
7                       The Alchemist - Paulo Coelho     254.0       budget   
8  Harry Potter 1–3 Box Set: A Magical Adventure ...    1030.0         high   
9                             One-Punch Man, Vol. 30     750.0          mid   

       category  days_since_added age_category  
0   Electronics                 0          new

In [29]:
# Create implicit feedback scores
event_weights = {
    'view': 1,
    'click': 2,
    'add_to_cart': 3,
    'purchase': 5
}

# Add weight column
events_df['weight'] = events_df['event_type'].map(event_weights)

# Aggregate interactions with weights
user_product_interactions = events_df.groupby(['user_id', 'product_id']).agg({
    'weight': 'sum',
    'created_at': 'max'  # Most recent interaction
}).reset_index()

user_product_interactions.columns = ['user_id', 'product_id', 'interaction_score', 'last_interaction_date']

# Add recency weight (decay factor)
days_since_interaction = (latest_date - user_product_interactions['last_interaction_date']).dt.days
recency_weight = np.exp(-days_since_interaction / 30)  # 30-day half-life

user_product_interactions['recency_weighted_score'] = (
    user_product_interactions['interaction_score'] * recency_weight
)

print("User-Product Interaction Matrix:")
print(user_product_interactions.head(10))
print(f"\nTotal interactions: {len(user_product_interactions)}")
print(f"Sparsity: {len(user_product_interactions) / (len(users_df) * len(products_df)) * 100:.2f}%")

User-Product Interaction Matrix:
   user_id  product_id  interaction_score   last_interaction_date  \
0        1           5                  7 2025-11-22 16:29:41.543   
1        1           7                  3 2025-12-04 18:43:08.378   
2        1           8                  2 2025-12-04 18:49:07.876   
3        1          12                 11 2025-11-22 16:33:15.388   
4        1          21                  3 2025-11-22 16:28:18.103   
5        1          31                  6 2025-12-04 18:46:18.548   
6        1          32                  1 2025-11-22 16:26:28.689   
7        1          33                  3 2025-12-07 21:28:28.650   
8        1          37                  3 2025-11-22 16:26:52.525   
9        1          43                  6 2025-11-22 16:27:10.465   

   recency_weighted_score  
0                3.145303  
1                2.010960  
2                1.340640  
3                4.942619  
4                1.347987  
5                4.021920  
6          

In [30]:
# Normalize numerical features for ML
from sklearn.preprocessing import StandardScaler

# Select numerical columns to normalize
user_numerical_cols = [
    'total_interactions', 'days_active', 'activity_rate', 'days_since_last_activity',
    'view_count', 'click_count', 'add_to_cart_count', 'purchase_count',
    'view_to_click_rate', 'click_to_cart_rate', 'overall_conversion_rate',
    'avg_purchase_price', 'avg_browsed_price', 'purchase_frequency'
]

product_numerical_cols = [
    'total_interactions', 'unique_users', 'view_count', 'click_count', 
    'add_to_cart_count', 'purchase_count', 'price',
    'click_through_rate', 'add_to_cart_rate', 'purchase_rate',
    'popularity_score_normalized', 'days_since_added'
]

# Create scaled versions
scaler_user = StandardScaler()
scaler_product = StandardScaler()

user_features_scaled = user_features.copy()
user_features_scaled[user_numerical_cols] = scaler_user.fit_transform(user_features[user_numerical_cols])

product_features_scaled = product_features.copy()
product_features_scaled[product_numerical_cols] = scaler_product.fit_transform(product_features[product_numerical_cols])

print("Features normalized")
print(f"\nUser features shape: {user_features_scaled.shape}")
print(f"Product features shape: {product_features_scaled.shape}")

Features normalized

User features shape: (101, 44)
Product features shape: (104, 22)


In [31]:
# Save to CSV files for later use
user_features.to_csv('user_features.csv', index=False)
product_features.to_csv('product_features.csv', index=False)
user_product_interactions.to_csv('user_product_interactions.csv', index=False)

# Save scaled versions
user_features_scaled.to_csv('user_features_scaled.csv', index=False)
product_features_scaled.to_csv('product_features_scaled.csv', index=False)

print("Features saved to CSV files:")
print("  user_features.csv")
print("  product_features.csv")
print("  user_product_interactions.csv")
print("  user_features_scaled.csv")
print("  product_features_scaled.csv")

Features saved to CSV files:
  user_features.csv
  product_features.csv
  user_product_interactions.csv
  user_features_scaled.csv
  product_features_scaled.csv


In [32]:
print("="*70)
print("FEATURE ENGINEERING SUMMARY")
print("="*70)

print("\nUSER FEATURES ({} features):".format(len(user_features.columns)))
print("  • Activity: total_interactions, days_active, activity_rate")
print("  • Engagement: view/click/cart/purchase counts")
print("  • Conversion: view_to_click_rate, overall_conversion_rate")
print("  • Category: favorite categories (1-3), category_diversity")
print("  • Price: avg_purchase_price, avg_browsed_price, price_preference")
print("  • Recency: days_since_last_activity, days_since_last_purchase")
print("  • Frequency: purchase_frequency")
print("  • Segment: user_segment (power_buyer, regular_buyer, etc.)")

print("\nPRODUCT FEATURES ({} features):".format(len(product_features.columns)))
print("  • Engagement: total_interactions, unique_users")
print("  • Events: view/click/cart/purchase counts")
print("  • Conversion: click_through_rate, purchase_rate")
print("  • Popularity: popularity_score, popularity_score_normalized")
print("  • Attributes: name, category, price, stock")
print("  • Price: price_bucket (budget/mid/high/luxury)")
print("  • Age: days_since_added, age_category")

print("\nINTERACTION MATRIX:")
print(f"  • Shape: {len(users_df)} users × {len(products_df)} products")
print(f"  • Total interactions: {len(user_product_interactions):,}")
print(f"  • Sparsity: {len(user_product_interactions) / (len(users_df) * len(products_df)) * 100:.2f}%")
print(f"  • Weighted scores: interaction_score, recency_weighted_score")

print("\nReady for Model Training!")
print("="*70)

FEATURE ENGINEERING SUMMARY

USER FEATURES (44 features):
  • Activity: total_interactions, days_active, activity_rate
  • Engagement: view/click/cart/purchase counts
  • Conversion: view_to_click_rate, overall_conversion_rate
  • Category: favorite categories (1-3), category_diversity
  • Price: avg_purchase_price, avg_browsed_price, price_preference
  • Recency: days_since_last_activity, days_since_last_purchase
  • Frequency: purchase_frequency
  • Segment: user_segment (power_buyer, regular_buyer, etc.)

PRODUCT FEATURES (22 features):
  • Engagement: total_interactions, unique_users
  • Events: view/click/cart/purchase counts
  • Conversion: click_through_rate, purchase_rate
  • Popularity: popularity_score, popularity_score_normalized
  • Attributes: name, category, price, stock
  • Price: price_bucket (budget/mid/high/luxury)
  • Age: days_since_added, age_category

INTERACTION MATRIX:
  • Shape: 101 users × 104 products
  • Total interactions: 4,047
  • Sparsity: 38.53%
  • Wei